## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [4]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 백만 도민의 나라로, 고대부터 현재까지 다양한 역사와 문화를 자랑한다. 주요 도시는 수도인 서울을 비롯해 부산, 대구, 인천 등이 있으며, 경제적으로는 세계적으로 강력한 식품, 기술 및 자동차 산업을 보유하고 있다. 한류 열풍으로 인해 한국 문화는 전 세계적으로 인기를 끌고 있으며, K-pop, K-drama, 한식 등이 세계 각지에서 사랑받고 있다. 한반도 북쪽에는 북한이 있으며, 남쪽에는 한국이 위치해 있으며, 두 나라 간에는 여전히 긴장 관계가 이어지고 있다. 현재는 코로나19 대응을 비롯한 다양한 사회적 문제들에 대처하면서 미래를 준비하고 있다.
CPU times: total: 62.5 ms
Wall time: 3.52 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [5]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도 경제 선진국가로, 세계적으로 유명한 문화와 역사를 가지고 있습니다. 수도는 서울이며, 국민의 대다수가 한국어를 사용합니다. 한국은 고대부터 현대까지 다양한 역사적 사건과 변화를 겪어왔으며, 현재는 선진화된 IT 기술과 엔터테인먼트 산업이 주목받고 있습니다. 또한, 한류 열풍으로 한국의 문화와 음식, 패션 등이 전 세계적으로 인기를 끌고 있습니다. 한반도 북쪽에는 북한이 위치하고 있으며, 남한과의 관계는 여전히 긴장 상태에 있지만 향후 평화적인 해결을 향해 노력하고 있습니다.
CPU times: total: 125 ms
Wall time: 3.32 s


In [6]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도 경제 선진국가로, 세계적으로 유명한 문화와 역사를 가지고 있습니다. 수도는 서울이며, 국민의 대다수가 한국어를 사용합니다. 한국은 고대부터 현대까지 다양한 역사적 사건과 변화를 겪어왔으며, 현재는 선진화된 IT 기술과 엔터테인먼트 산업이 주목받고 있습니다. 또한, 한류 열풍으로 한국의 문화와 음식, 패션 등이 전 세계적으로 인기를 끌고 있습니다. 한반도 북쪽에는 북한이 위치하고 있으며, 남한과의 관계는 여전히 긴장 상태에 있지만 향후 평화적인 해결을 향해 노력하고 있습니다.
CPU times: total: 0 ns
Wall time: 0 ns


## SQLite Cache


In [7]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [8]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 현대국가로, 역사적으로는 5000년 이상의 역사를 자랑한다. 경제적으로는 세계적으로 주요한 경제강국으로 발전하였고, 기술력과 문화적 영향력도 크게 증가하였다. 한국은 또한 한류와 K-pop을 통해 세계 각지에서 큰 인기를 얻고 있으며, 한국 요리와 전통 문화도 많은 사람들에게 사랑받고 있다. 정치적으로는 북한과의 관계가 계속 이슈가 되고 있으며, 미국과의 동맹 관계도 중요하게 다뤄지고 있다. 한국은 다양한 측면에서 세계에서 주목받는 나라 중 하나이다.
CPU times: total: 0 ns
Wall time: 5.92 s


In [9]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 고도로 발전한 현대국가로, 역사적으로는 5000년 이상의 역사를 자랑한다. 경제적으로는 세계적으로 주요한 경제강국으로 발전하였고, 기술력과 문화적 영향력도 크게 증가하였다. 한국은 또한 한류와 K-pop을 통해 세계 각지에서 큰 인기를 얻고 있으며, 한국 요리와 전통 문화도 많은 사람들에게 사랑받고 있다. 정치적으로는 북한과의 관계가 계속 이슈가 되고 있으며, 미국과의 동맹 관계도 중요하게 다뤄지고 있다. 한국은 다양한 측면에서 세계에서 주목받는 나라 중 하나이다.
CPU times: total: 78.1 ms
Wall time: 167 ms
